In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
train_data = pd.read_csv('ratingsTrain.txt', header=0, delimiter='\t', quoting=3)
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [3]:
print(train_data['document'][123], train_data['label'][123])

별점10점가자 1


In [4]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 75.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 12.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.6 MB/s eta 0:00:00:00:01


In [5]:
from konlpy.tag import Okt
import re

In [6]:
# 감탄사, 조사들은 제거할 수 있도록 정의
stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한'])

okt = Okt()


In [7]:
def preprocessing(review, okt, remove_stopwords=False, stop_words=None):
    if stop_words is None:
        stop_words = set()

    try:
        # 문장에서 한글만 뽑아냄
        review_text = re.sub("[^가-힣ㄱ-하-ㅣ\\s]", "", review)

        # okt 라이브러리로 단어의 어간별로 분리
        word_review = okt.morphs(review_text, stem=True)

        # 감탄사와 조사 제거 - (suppose) 감탄사와 조사는 morphs가 분리시켜주지 않는 것으로 보임
        if remove_stopwords:
            word_review = [token for token in word_review if token not in stop_words]

        return word_review

    except Exception as e:
        print(f"An error occurred during preprocessing: {e}")
        return None

# 예시 사용
review = "오늘은 정말 좋은 날씨네요!"
preprocessed_review = preprocessing(review, okt, remove_stopwords=True, stop_words=stop_words)

if preprocessed_review is not None:
    print(preprocessed_review)


['오늘', '정말', '좋다', '날씨', '네', '요']


In [8]:
# 이상한 데이터 정제
clean_train_review = []

for review in train_data['document']:
    if type(review) == str:
        clean_train_review.append(preprocessing(review, okt, True, stop_words))
    else:
        clean_train_review.append([])

In [9]:
clean_train_review[:4]

[['더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정']]

### Text to Number

In [10]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

# 전체 순서를 만들고
tokenizer.fit_on_texts(clean_train_review)

# 단어의 위치를 숫자로 표시
train_sequence = tokenizer.texts_to_sequences(clean_train_review)

MAX_SEQUENCE_LENGTH = 8

# 최대 문장 길이는 8로 padding
train_inputs = pad_sequences(train_sequence, maxlen = MAX_SEQUENCE_LENGTH, padding='post')

# train label 만들기
train_labels = np.array(train_data['label'])

In [11]:
train_inputs[:4]

array([[  466,    20,   266,   666,     0,     0,     0,     0],
       [  606,     1,   220,  1461,    30,   971,   682,    24],
       [  395,  2458, 25061,  2325,  5682,     2,   227,    13],
       [ 6508,   110,  8143,   225,    61,     8,    31,  3623]],
      dtype=int32)

In [16]:
# 10%는 Eval Data로 사용
TEST_SPLIT = 0.1
RNG_SEED = 13371447

# 데이터를 train과 eval로 분리
from sklearn.model_selection import train_test_split
input_train, input_eval, label_train, label_eval = train_test_split(train_inputs, train_labels, test_size=TEST_SPLIT, random_state=RNG_SEED)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding, Conv1D

model = Sequential()

# Word Embedding
word_size = len(tokenizer.word_index)+1
model.add(Embedding(word_size, 128, input_length=MAX_SEQUENCE_LENGTH))

# DNN Training, Activation은 relu
model.add(Flatten())
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# epoch을 5로 해서 간단히 학습
model.fit(input_train, label_train, epochs=5)

Epoch 1/5
4219/4219 [==============================] - 334s 79ms/step - loss: 0.6057 - accuracy: 0.7581
Epoch 2/5
4219/4219 [==============================] - 329s 78ms/step - loss: 0.4725 - accuracy: 0.8387
Epoch 3/5
4219/4219 [==============================] - 328s 78ms/step - loss: 0.4266 - accuracy: 0.8702
Epoch 4/5
4219/4219 [==============================] - 328s 78ms/step - loss: 0.4550 - accuracy: 0.8822
Epoch 5/5
4219/4219 [==============================] - 327s 78ms/step - loss: 0.4135 - accuracy: 0.8995


In [17]:
model.evaluate(input_eval, label_eval)

469/469 [==============================] - 1s 1ms/step - loss: 1.1693 - accuracy: 0.7368


[1.1692878007888794, 0.7368000149726868]

In [23]:
def predict_review(sentence, model):
    # 테스트 문장을 전처리
    test_prepro = preprocessing(sentence, okt, True, stop_words)
    test_review = []
    test_review.append(test_prepro)
    
    # 전처리된 문장을 토큰화
    test_token = tokenizer.texts_to_sequences(test_review)
    test_seq = pad_sequences(test_token, maxlen = MAX_SEQUENCE_LENGTH, padding='post')
    ret = model.predict(test_seq)
    if ret > 0.5:
        sentiment = "긍정"
    elif ret < 0.5:
        sentiment = "부정"
    elif ret == 0.5:
        sentiment = "중립"
    
    return sentiment

In [31]:
# 테스트
predict_review('', model)

1/1 [==============================] - 0s 22ms/step


'부정'

In [33]:
from keras.models import load_model
model.save('textSA_ver1.keras')